# Introduction
In this colab notebook we describe our solution to exercise 0 in Autonomous Robots course.

The final goal is to calculate location using raw data from a GPS receiver.

After each step, all output files can be accessed using Colab files tab.

Output files:
- Parsed android gnss log file (csv) - '2_parsed_gps.csv'
- Sat calculated locations with satellite positions (csv) - '5_final_output.csv'
- KML file with the calculated location points - '5_points.kml'

# Preparation

## Download the dataset
Downloading the dataset from google drive folder and extract the zip files

In [1]:
!pip install gdown -q
# Download the dataset from the Google Drive folder and unzip
!gdown --folder https://drive.google.com/drive/folders/1qZ8URVwwjrbTf_sDTgKoenw0OnwZKh1X
%cd Ex0
!unzip -qo Driving.zip -d Driving
!unzip -qo Fixed.zip -d Fixed
!unzip -qo Walking.zip -d Walking
%cd /content

Retrieving folder contents
Processing file 1lY9-gI8xzulCvgvFHeMGCYurIFybb8Ao Driving.zip
Processing file 1yz97v51nHk67y1y75azRPMrf81FcozI3 Fixed.zip
Processing file 17zirF2lJ2F4lp3qq-51fDTBsxUA4p_SG Walking.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1lY9-gI8xzulCvgvFHeMGCYurIFybb8Ao
To: /content/Ex0/Driving.zip
100% 715k/715k [00:00<00:00, 101MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yz97v51nHk67y1y75azRPMrf81FcozI3
To: /content/Ex0/Fixed.zip
100% 85.8k/85.8k [00:00<00:00, 69.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=17zirF2lJ2F4lp3qq-51fDTBsxUA4p_SG
To: /content/Ex0/Walking.zip
100% 251k/251k [00:00<00:00, 78.0MB/s]
Download completed
/content/Ex0
/content


## Install requirements

In [2]:
!wget https://raw.githubusercontent.com/johnsonmitchelld/gnss-analysis/main/gnssutils/ephemeris_manager.py  -P /content/gnssutils/
!pip install -q georinex unlzw3 simplekml folium
!pip install -q "pandas<2.0"

--2024-05-15 19:56:45--  https://raw.githubusercontent.com/johnsonmitchelld/gnss-analysis/main/gnssutils/ephemeris_manager.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9025 (8.8K) [text/plain]
Saving to: ‘/content/gnssutils/ephemeris_manager.py’

ephemeris_manager.p 100%[===================>]   8.81K  --.-KB/s    in 0s      

2024-05-15 19:56:46 (55.8 MB/s) - ‘/content/gnssutils/ephemeris_manager.py’ saved [9025/9025]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB

In [3]:
# IMPORTS
import pandas as pd
pd.options.mode.copy_on_write = True
from datetime import datetime, timezone, timedelta
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from gnssutils.ephemeris_manager import EphemerisManager

# **Solution**

* Need to choose example log file for testing the solution

In [26]:
# @title Select file to parse { run: "auto" }

file_type = "Driving" # @param ["Fixed", "Driving", "Walking"]

filepath_map = {"Fixed": '/content/Ex0/Fixed/gnss_log_2024_04_13_19_51_17.txt',
                "Driving": '/content/Ex0/Driving/gnss_log_2024_04_13_19_53_33.txt',
                "Walking": '/content/Ex0/Walking/gnss_log_2024_04_13_19_52_00.txt'}

file_to_parse = filepath_map[file_type]
print(f'File to parse: "{file_to_parse}"')

File to parse: "/content/Ex0/Driving/gnss_log_2024_04_13_19_53_33.txt"


In [ ]:
## Can be uncomment for use custom file to parse
# file_to_parse = 'custom/file/path'

## **2. Parsing tool**
Converting the Android GNSS log file to a CSV file

In [27]:
WEEKSEC = 604800
LIGHTSPEED = 2.99792458e8


def load_gnss_log_raw_data(file_path):
  raw = []
  fix = []

  with open(file_path) as gnss_log:
    line_content = gnss_log.readline()
    while line_content:
      if line_content.startswith('# Raw') or line_content.startswith('Raw,'):
        raw.append(line_content.replace('\n', '').split(','))
      if line_content.startswith('# Fix') or line_content.startswith('Fix,'):
        fix.append(line_content.replace('\n', '').split(','))

      line_content = gnss_log.readline()

  raw_df = pd.DataFrame(raw[1:], columns=raw[0])
  fix_df = pd.DataFrame(fix[1:], columns=fix[0])

  return raw_df, fix_df


def parse_raw_data(measurements):

  # Remove all non-GPS measurements
  measurements = measurements[measurements['ConstellationType'] == '1']

  # Format satellite IDs
  measurements.loc[measurements['Svid'].str.len() != 1, 'SvName'] = 'G' + measurements['Svid']
  measurements.loc[measurements['Svid'].str.len() == 1, 'SvName'] = 'G0' + measurements['Svid']

  # Convert columns to numeric representation
  measurements['Cn0DbHz'] = pd.to_numeric(measurements['Cn0DbHz'])
  measurements['TimeNanos'] = pd.to_numeric(measurements['TimeNanos'])
  measurements['FullBiasNanos'] = pd.to_numeric(measurements['FullBiasNanos'])
  measurements['ReceivedSvTimeNanos']  = pd.to_numeric(measurements['ReceivedSvTimeNanos'])
  measurements['PseudorangeRateMetersPerSecond'] = pd.to_numeric(measurements['PseudorangeRateMetersPerSecond'])
  measurements['ReceivedSvTimeUncertaintyNanos'] = pd.to_numeric(measurements['ReceivedSvTimeUncertaintyNanos'])

  # A few measurement values are not provided by all phones
  # We'll check for them and initialize them with zeros if missing
  if 'BiasNanos' in measurements.columns:
      measurements['BiasNanos'] = pd.to_numeric(measurements['BiasNanos'])
  else:
      measurements['BiasNanos'] = 0
  if 'TimeOffsetNanos' in measurements.columns:
      measurements['TimeOffsetNanos'] = pd.to_numeric(measurements['TimeOffsetNanos'])
  else:
      measurements['TimeOffsetNanos'] = 0

  # Add GPStime
  measurements['GpsTimeNanos'] = measurements['TimeNanos'] - (measurements['FullBiasNanos'] - measurements['BiasNanos'])
  gpsepoch = datetime(1980, 1, 6, 0, 0, 0)
  measurements['UnixTime'] = pd.to_datetime(measurements['GpsTimeNanos'], utc = True, origin=gpsepoch)
  measurements['UnixTime'] = measurements['UnixTime']

  # Split data into measurement epochs
  measurements['Epoch'] = 0
  measurements.loc[measurements['UnixTime'] - measurements['UnixTime'].shift() > timedelta(milliseconds=200), 'Epoch'] = 1
  measurements['Epoch'] = measurements['Epoch'].cumsum()

  # This should account for rollovers since it uses a week number specific to each measurement

  measurements['tRxGnssNanos'] = measurements['TimeNanos'] + measurements['TimeOffsetNanos'] - (measurements['FullBiasNanos'].iloc[0] + measurements['BiasNanos'].iloc[0])
  measurements['GpsWeekNumber'] = np.floor(1e-9 * measurements['tRxGnssNanos'] / WEEKSEC)
  measurements['tRxSeconds'] = 1e-9*measurements['tRxGnssNanos'] - WEEKSEC * measurements['GpsWeekNumber']
  measurements['tTxSeconds'] = 1e-9*(measurements['ReceivedSvTimeNanos'] + measurements['TimeOffsetNanos'])
  # Calculate pseudorange in seconds
  measurements['prSeconds'] = measurements['tRxSeconds'] - measurements['tTxSeconds']

  # Conver to meters
  measurements['PrM'] = LIGHTSPEED * measurements['prSeconds']
  measurements['PrSigmaM'] = LIGHTSPEED * 1e-9 * measurements['ReceivedSvTimeUncertaintyNanos']

  return measurements


def calculate_satellite_position(ephemeris, transmit_time):
    mu = 3.986005e14
    OmegaDot_e = 7.2921151467e-5
    F = -4.442807633e-10
    sv_position = pd.DataFrame()
    sv_position['sv']= ephemeris.index
    sv_position.set_index('sv', inplace=True)
    sv_position['t_k'] = transmit_time - ephemeris['t_oe']
    A = ephemeris['sqrtA'].pow(2)
    n_0 = np.sqrt(mu / A.pow(3))
    n = n_0 + ephemeris['deltaN']
    M_k = ephemeris['M_0'] + n * sv_position['t_k']
    E_k = M_k
    err = pd.Series(data=[1]*len(sv_position.index))
    i = 0
    while err.abs().min() > 1e-8 and i < 10:
        new_vals = M_k + ephemeris['e']*np.sin(E_k)
        err = new_vals - E_k
        E_k = new_vals
        i += 1

    sinE_k = np.sin(E_k)
    cosE_k = np.cos(E_k)
    delT_r = F * ephemeris['e'].pow(ephemeris['sqrtA']) * sinE_k
    delT_oc = transmit_time - ephemeris['t_oc']
    sv_position['delT_sv'] = ephemeris['SVclockBias'] + ephemeris['SVclockDrift'] * delT_oc + ephemeris['SVclockDriftRate'] * delT_oc.pow(2)

    v_k = np.arctan2(np.sqrt(1-ephemeris['e'].pow(2))*sinE_k,(cosE_k - ephemeris['e']))

    Phi_k = v_k + ephemeris['omega']

    sin2Phi_k = np.sin(2*Phi_k)
    cos2Phi_k = np.cos(2*Phi_k)

    du_k = ephemeris['C_us']*sin2Phi_k + ephemeris['C_uc']*cos2Phi_k
    dr_k = ephemeris['C_rs']*sin2Phi_k + ephemeris['C_rc']*cos2Phi_k
    di_k = ephemeris['C_is']*sin2Phi_k + ephemeris['C_ic']*cos2Phi_k

    u_k = Phi_k + du_k

    r_k = A*(1 - ephemeris['e']*np.cos(E_k)) + dr_k

    i_k = ephemeris['i_0'] + di_k + ephemeris['IDOT']*sv_position['t_k']

    x_k_prime = r_k*np.cos(u_k)
    y_k_prime = r_k*np.sin(u_k)

    Omega_k = ephemeris['Omega_0'] + (ephemeris['OmegaDot'] - OmegaDot_e)*sv_position['t_k'] - OmegaDot_e*ephemeris['t_oe']

    sv_position['x_k'] = x_k_prime*np.cos(Omega_k) - y_k_prime*np.cos(i_k)*np.sin(Omega_k)
    sv_position['y_k'] = x_k_prime*np.sin(Omega_k) + y_k_prime*np.cos(i_k)*np.cos(Omega_k)
    sv_position['z_k'] = y_k_prime*np.sin(i_k)
    return sv_position


In [28]:
raw_data, fix_data = load_gnss_log_raw_data(file_to_parse)

measurements = parse_raw_data(raw_data)
measurements

,# Raw,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,...,GpsTimeNanos,UnixTime,Epoch,tRxGnssNanos,GpsWeekNumber,tRxSeconds,tTxSeconds,prSeconds,PrM,PrSigmaM
0,Raw,1713027213426,332431420000000,18,0.0,-1396730000006041643,-0.135250,15.701487427577376,-34.49754549238822,10.698025679382875,...,1.397062e+18,2024-04-13 16:53:51.426041600+00:00,0,1.397062e+18,2309.0,579231.426042,579231.349849,0.076193,2.284207e+07,1.798755
1,Raw,1713027213426,332431420000000,18,0.0,-1396730000006041643,-0.135250,15.701487427577376,-34.49754549238822,10.698025679382875,...,1.397062e+18,2024-04-13 16:53:51.426041600+00:00,0,1.397062e+18,2309.0,579231.426042,579231.353770,0.072272,2.166653e+07,2.698132
2,Raw,1713027213426,332431420000000,18,0.0,-1396730000006041643,-0.135250,15.701487427577376,-34.49754549238822,10.698025679382875,...,1.397062e+18,2024-04-13 16:53:51.426041600+00:00,0,1.397062e+18,2309.0,579231.426042,579231.351922,0.074120,2.222064e+07,5.696057
3,Raw,1713027213426,332431420000000,18,0.0,-1396730000006041643,-0.135250,15.701487427577376,-34.49754549238822,10.698025679382875,...,1.397062e+18,2024-04-13 16:53:51.426041600+00:00,0,1.397062e+18,2309.0,579231.426042,579231.354381,0.071661,2.148338e+07,1.798755
25,Raw,1713027213426,332431420000000,18,0.0,-1396730000006041643,-0.135250,15.701487427577376,-34.49754549238822,10.698025679382875,...,1.397062e+18,2024-04-13 16:53:51.426041600+00:00,0,1.397062e+18,2309.0,579231.426042,579231.355459,0.070583,2.116013e+07,4.197094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9172,Raw,1713027387426,332605420000000,18,0.0,-1396730000006043192,-0.349374,30.755258194403723,-1.3494617332354533,20.89038448284447,...,1.397063e+18,2024-04-13 16:56:45.426043136+00:00,174,1.397063e+18,2309.0,579405.426042,579405.354209,0.071833,2.153501e+07,2.098547
9192,Raw,1713027387426,332605420000000,18,0.0,-1396730000006043192,-0.349374,30.755258194403723,-1.3494617332354533,20.89038448284447,...,1.397063e+18,2024-04-13 16:56:45.426043136+00:00,174,1.397063e+18,2309.0,579405.426042,579405.355326,0.070715,2.119991e+07,3.297717
9193,Raw,1713027387426,332605420000000,18,0.0,-1396730000006043192,-0.349374,30.755258194403723,-1.3494617332354533,20.89038448284447,...,1.397063e+18,2024-04-13 16:56:45.426043136+00:00,174,1.397063e+18,2309.0,579405.426042,579405.349627,0.076415,2.290851e+07,2.398340
9194,Raw,1713027387426,332605420000000,18,0.0,-1396730000006043192,-0.349374,30.755258194403723,-1.3494617332354533,20.89038448284447,...,1.397063e+18,2024-04-13 16:56:45.426043136+00:00,174,1.397063e+18,2309.0,579405.426042,579405.351590,0.074452,2.232008e+07,8.094396


In [29]:
# We used this manager to get the data from the nasa db for the satellites location
manager = EphemerisManager()

rows = []

for epoch in measurements['Epoch'].unique():

    one_epoch = measurements.loc[(measurements['Epoch'] == epoch) & (measurements['prSeconds'] < 0.1)]
    one_epoch.drop_duplicates(subset='SvName', inplace=True)
    one_epoch.set_index('SvName', inplace=True)

    if len(one_epoch) < 5: # Not enough satellites for location -> skip this epoch
      continue

    timestamp = one_epoch.iloc[0]['UnixTime'].to_pydatetime(warn=False)

    sats = one_epoch.index.unique().tolist()
    ephemeris = manager.get_ephemeris(timestamp, sats)

    sat_pos = calculate_satellite_position(ephemeris=ephemeris, transmit_time=one_epoch['tTxSeconds'])

    for sv in sat_pos.index:
      # GPS time, SatPRN (ID), Sat.X, Sat.Y, Sat.Z, Pseudo-Range, CN0
      rows.append({
          "GPS Time": timestamp.isoformat(),
          "SatPRN (ID)": sv,
          "Sat.X": sat_pos.at[sv, 'x_k'],
          "Sat.Y": sat_pos.at[sv, 'y_k'],
          "Sat.Z": sat_pos.at[sv, 'z_k'],
          "Pseudo-Range": one_epoch.at[sv, 'PrM'] + (LIGHTSPEED * sat_pos.at[sv, 'delT_sv']),
          "CN0": one_epoch.at[sv, 'Cn0DbHz'],
      })

# Save the parsed data of the satellites locations
parsed_gps = pd.DataFrame(rows)
parsed_gps.to_csv('2_parsed_gps.csv',index=False)

## **3. Positioning Algorithm**

The algorithm estimates the position of a GPS receiver using raw data from multiple satellites, leveraging the least squares optimization method. The core of the algorithm is a weighted least squares optimization, where the differences between observed and calculated pseudo-ranges are minimized.
values are weighted by the square root of the normalized CN0 values, giving more importance to higher quality signals. The result of the algorithm provides an estimated ECEF position (X, Y, Z) of the GPS receiver and the clock bias.








In [30]:
from scipy.optimize import least_squares

def calc_position(sat_positions, pseudo_ranges, weights):

    # Initial guess for the receiver's position (X, Y, Z, clock bias)
    x0 = np.array([0, 0, 0, 0])

    def weighted_residuals(x):
        """ Calculate weighted residuals for the least squares solver """
        distances = np.sqrt(np.sum((sat_positions - x[:3])**2, axis=1))
        residuals = distances + x[3] - pseudo_ranges
        return residuals * np.sqrt(weights)  # Apply square root of weights

    # Use least squares optimization to solve for the position
    result = least_squares(weighted_residuals, x0)

    # Extract the estimated position
    estimated_position = result.x[:3]
    clock_bias = result.x[3]

    return estimated_position, clock_bias

## **4. Convert ECEF to LLA function**

In [31]:
import pyproj

def ecefTolla(ecef):
  transformer = pyproj.Transformer.from_crs(
      {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
      {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
    )

  lon1, lat1, alt1 = transformer.transform(ecef[0],ecef[1],ecef[2],radians=False)
  return (lat1, lon1, alt1)

## **5. Calculate the Position using the algorithm**

In [32]:
# Use the data of the sat location from the previous step
sat_loc = parsed_gps
# Can also read the satellites locations also from the csv we saved in section 2
# sat_loc = pd.read_csv('2_parsed_gps.csv')

calculated_locs = []

for timestemp in sat_loc['GPS Time'].unique():
  current_epoch = sat_loc[sat_loc['GPS Time'] == timestemp]
  # Use only the top 5 satellites
  current_epoch = current_epoch.nlargest(5, 'CN0')

  sat_pos = current_epoch[['Sat.X', 'Sat.Y', 'Sat.Z']].to_numpy()
  pr = current_epoch['Pseudo-Range'].to_numpy()
  # Convert CN0 to weights
  w = current_epoch['CN0'].to_numpy()
  w = w / w.sum()

  ecef, bias = calc_position(sat_pos, pr,  w)
  lla = ecefTolla(ecef)
  # if bias < -1000: continue
  # Pos.X, Pos.Y, Pos,Z, Lat, Lon, Alt
  calculated_locs.append({'time': timestemp,
                          'Pos.X': ecef[0],
                          'Pos.Y': ecef[1],
                          'Pos.Z': ecef[2],
                          'Lat': lla[0],
                          'Lon': lla[1],
                          'Alt': lla[2],
                          'Bias': bias,
                          })

calc_loc = pd.DataFrame(calculated_locs)

In [33]:
# Merging the data of the sat location and the calculated positions to one table
merged_data = sat_loc.join(calc_loc.set_index('time'), on='GPS Time')
merged_data.set_index('GPS Time', inplace=True)
merged_data.to_csv('5_final_output.csv')
merged_data

,SatPRN (ID),Sat.X,Sat.Y,Sat.Z,Pseudo-Range,CN0,Pos.X,Pos.Y,Pos.Z,Lat,Lon,Alt,Bias
GPS Time,,,,,,,,,,,,,
2024-04-13T16:53:51.426041+00:00,G08,2.502364e+07,4.783846e+06,8.137692e+06,2.119601e+07,27.7,4.438037e+06,3.085303e+06,3.376633e+06,32.166618,34.806853,1028.453239,287.025845
2024-04-13T16:53:51.426041+00:00,G10,-1.620082e+06,1.732768e+07,2.017249e+07,2.283953e+07,48.3,4.438037e+06,3.085303e+06,3.376633e+06,32.166618,34.806853,1028.453239,287.025845
2024-04-13T16:53:51.426041+00:00,G21,1.575745e+07,1.890976e+06,2.185636e+07,2.170471e+07,45.7,4.438037e+06,3.085303e+06,3.376633e+06,32.166618,34.806853,1028.453239,287.025845
2024-04-13T16:53:51.426041+00:00,G27,2.309844e+07,1.330337e+07,-3.014966e+06,2.221512e+07,37.9,4.438037e+06,3.085303e+06,3.376633e+06,32.166618,34.806853,1028.453239,287.025845
2024-04-13T16:53:51.426041+00:00,G32,7.810469e+06,1.784981e+07,1.835083e+07,2.129809e+07,48.5,4.438037e+06,3.085303e+06,3.376633e+06,32.166618,34.806853,1028.453239,287.025845
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-13T16:56:45.426043+00:00,G08,2.516640e+07,4.893989e+06,7.630018e+06,2.123814e+07,28.0,4.436882e+06,3.085269e+06,3.376323e+06,32.168900,34.813544,44.214596,-458.650429
2024-04-13T16:56:45.426043+00:00,G10,-1.989091e+06,1.756017e+07,1.992916e+07,2.290833e+07,42.1,4.436882e+06,3.085269e+06,3.376323e+06,32.168900,34.813544,44.214596,-458.650429
2024-04-13T16:56:45.426043+00:00,G21,1.583583e+07,2.345511e+06,2.177063e+07,2.165212e+07,33.5,4.436882e+06,3.085269e+06,3.376323e+06,32.168900,34.813544,44.214596,-458.650429


**Create KML file from the locations**

In [34]:
import simplekml
kml = simplekml.Kml()

for point in calculated_locs:
  kml.newpoint(name=point['time'], coords=[(point['Lon'], point['Lat'], point['Alt'])])  # lon, lat optional height

kml.save('5_points.kml')

# **Display locations on map**
Can compare the locations that calculated using our algorithm to the fix locations got from the android using the layer control.

Raw - Blue markers

Fix - Green markers

In [35]:
import folium
fig = folium.Map(location = (calculated_locs[0]['Lat'], calculated_locs[0]['Lon']),zoom_start = 16)

# Add the Raw points
raw_group = folium.FeatureGroup(name="Raw").add_to(fig)
for loc in calculated_locs:
  text = f"Lattitude:<br>{loc['Lat']}<br>Longitude:<br>{loc['Lon']}<br>time:<br>{loc['time']}<br>bias:<br>{loc['Bias']}"
  # if loc['Bias'] > -1000:
  folium.Marker((loc['Lat'], loc['Lon']), popup=text, icon=folium.Icon(color='blue')).add_to(raw_group)

# Add the Fix points
fix_group = folium.FeatureGroup(name="Fix", show=False).add_to(fig)
for i, r in fix_data[fix_data['Provider'] == 'GPS'].iterrows():
  folium.Marker((r['LatitudeDegrees'], r['LongitudeDegrees']), icon=folium.Icon(color='lightgreen')).add_to(fix_group)

folium.LayerControl().add_to(fig)

fig